In [3]:
%matplotlib inline

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    Geometry,
    SentinelHubStatistical,
    SentinelHubStatisticalDownloadClient,
    SHConfig,
    parse_time,
)

In [15]:
config = SHConfig("pip")
byoc_id = "ccb1f8f0-e5bf-4c31-afe5-d8803bcbde2a"

In [6]:
# load polygons.geojson file
polygons = gpd.read_file("polygons.geojson")

In [25]:
bounds = BBox(polygons.geometry[0].bounds, CRS.WGS84)

rgb_evalscript = """
//VERSION=3

function setup() {
  return {
    input: [
      {
        bands: [
          "coastal_blue",
          "blue",
          "green_i",
          "green",
          "yellow",
          "red",
          "rededge",
          "nir",
          "dataMask"
        ]
      }
    ],
    output: [
      {
        id: "rgb",
        bands: [
          "coastal_blue",
          "blue",
          "green_i",
          "green",
          "yellow",
          "red",
          "rededge",
          "nir"
        ]
      },
      {
        id: "dataMask",
        bands: 1
      }
    ]
  }
}

function evaluatePixel(samples) {
    return {
      rgb: [
        samples.coastal_blue,
        samples.blue,
        samples.green_i,
        samples.green,
        samples.yellow,
        samples.red,
        samples.rededge,
        samples.nir
      ],
      dataMask: [samples.dataMask]
    };
}
"""
requests = []

for geo_shape in polygons.geometry.values:
  bounds = BBox(geo_shape.bounds, CRS.WGS84)
  request = SentinelHubStatistical(
    aggregation=SentinelHubStatistical.aggregation(
        evalscript=rgb_evalscript,
        time_interval=("2022-05-01", "2023-05-01"),
        aggregation_interval="P1M",
        size=(512, 512),
    ),
    input_data=[SentinelHubStatistical.input_data(DataCollection.define_byoc(byoc_id))],
    bbox=bounds,
    config=config,
  )
  requests.append(request)


In [28]:
download_requests = [request.download_list[0] for request in requests]

client = SentinelHubStatisticalDownloadClient(config=config)

stats = client.download(download_requests)

In [66]:
df = pd.json_normalize([stat["data"][0] for stat in stats])
df = df[[column for column in df.columns if column.endswith(("mean", "stDev"))]]

In [71]:
band_stats = df.mean()
band_stats

outputs.rgb.bands.coastal_blue.stats.mean     1720.914975
outputs.rgb.bands.coastal_blue.stats.stDev     747.259017
outputs.rgb.bands.blue.stats.mean             1715.600931
outputs.rgb.bands.blue.stats.stDev             698.575382
outputs.rgb.bands.green_i.stats.mean          1913.451187
outputs.rgb.bands.green_i.stats.stDev          739.039452
outputs.rgb.bands.green.stats.mean            2088.540235
outputs.rgb.bands.green.stats.stDev            768.436933
outputs.rgb.bands.yellow.stats.mean           2274.610611
outputs.rgb.bands.yellow.stats.stDev           849.646913
outputs.rgb.bands.red.stats.mean              2290.037877
outputs.rgb.bands.red.stats.stDev              868.948089
outputs.rgb.bands.rededge.stats.mean          2613.193389
outputs.rgb.bands.rededge.stats.stDev          849.233234
outputs.rgb.bands.nir.stats.mean              3970.674317
outputs.rgb.bands.nir.stats.stDev              914.576073
dtype: float64

In [77]:
# select all elements in band_stats that end with .mean
means = band_stats[band_stats.index.str.endswith(".mean")]
means.index = means.index.str.split(".").str[-3]
dict(means)

{'coastal_blue': 1720.9149748636517,
 'blue': 1715.6009309663668,
 'green_i': 1913.451186699416,
 'green': 2088.540234590639,
 'yellow': 2274.6106106078814,
 'red': 2290.0378766752215,
 'rededge': 2613.193388683956,
 'nir': 3970.6743174860667}

In [79]:
# select all elements in band_stats that end with .stDev
stds = band_stats[band_stats.index.str.endswith(".stDev")]
stds.index = stds.index.str.split(".").str[-3]
dict(stds)

{'coastal_blue': 747.2590169771133,
 'blue': 698.5753822151314,
 'green_i': 739.0394519018009,
 'green': 768.4369332722395,
 'yellow': 849.6469129759253,
 'red': 868.9480887199492,
 'rededge': 849.2332344248075,
 'nir': 914.5760734235755}